In [63]:
import numpy as np
from scipy.stats import ortho_group

In [64]:
y = np.array([1, 2, 3])  # input vector

In [65]:
N = 2       #   No. of basis functions
K = 3      #   Dimensions of the input vector
gamma = .5
active_set = set()
features = np.zeros(N)
signs = np.zeros(N, dtype='int8')
A = ortho_group.rvs(dim=K)    # Matrix containing orthogonal bases
A = A[:,:N]
#(A[:,:N].T).dot(A[:,:N])

In [66]:
part_diff = np.zeros(N)
for i in range(N):
    for j in range(K):
        part_diff[i] += -2*(y[j] - A[j, :].dot(features))*A[j, i]
max_val = np.amax(part_diff)
max_ind = np.argmax(part_diff)
if(max_val > gamma):
    signs[max_ind] = -1
    active_set = active_set | {max_ind}
if(max_val < -gamma):
    signs[max_ind] = 1
    active_set = active_set | {max_ind}


In [89]:
A_hat = A[:, list(active_set)]
features_hat = features[list(active_set)]
signs_hat = signs[list(active_set)]
features_hat_new = np.linalg.inv(A_hat.T @ A_hat) @ ((A_hat.T @ y) - (gamma/2)*signs_hat)

In [90]:
features_hat_new

array([-0.21620289])

14